<a href="https://colab.research.google.com/github/planchon/tweeter_fakenews/blob/main/Tweeter_%26_Fake_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tweeter & Fake news
Notebook jupyter du projet tweeter et fake news par Arnaud Valette & Paul Planchon

In [ ]:
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
print("commit test")

commit test
